In [6]:
import pandas as pd

# Basic usage
df = pd.read_excel('/home/ubuntu/bgg/evals-2.xlsx')

# With more options
# df = pd.read_excel(
#     'your_file.xlsx',
#     sheet_name='Sheet1',  # Specify sheet name or index (0 is first sheet)
#     header=0,             # Row to use as column names (0-indexed)
#     usecols='A:D',        # Columns to read (can be range, list of indices, or names)
#     skiprows=2,           # Skip first 2 rows
#     nrows=100             # Read only 100 rows
# )

# Print the first few rows to verify
print(df)

   "Categogy (Used by Qwen2.5, Falcon3 7B, DeepSeek)"  Shots  \
0                                       General Tasks    NaN   
1                                                MMLU      5   
2                                                MMLU      0   
3                                            MMLU-Pro      5   
4                                          MMLU-redux      5   
5                                                 BBH      3   
6                                               ARC-C      0   
7                                          TruthfulQA      0   
8                                          Winogrande      5   
9                                           HellaSwag      0   
10                                            AGIEval      0   
11                                               DROP      3   
12                                           Triviaqa      5   
13                                               PIQA      0   
14                                      

In [ ]:
import re
import pandas as pd

# -------------------------------------------------
# 1.  Load / point to your dataframe
# -------------------------------------------------
# df = pd.read_csv("your_file.csv")      # ⬅ or reuse the frame already in RAM

# -------------------------------------------------
# 2.  Helper: model‑name → (size_in_B, group_name)
# -------------------------------------------------
def parse_model_name(name: str):
    m = re.search(r"([\d.]+)\s*([BM])", name, re.I)
    size = None
    if m:
        num, unit = m.groups()
        size = float(num)
        if unit.upper() == "M":          # convert M ➜ billions
            size /= 1_000

    # group = everything before the first digit or space‐digit combo
    group = re.split(r"\s|-(?=\d)", name.strip())[0]
    return size, group


# -------------------------------------------------
# 3.  Build the benchmark‑row index automatically
# -------------------------------------------------
first_col = df.columns[0]

all_benchmarks = (
    df[first_col]
    .dropna()
    .loc[lambda s: ~s.str.endswith("Tasks")]      # drop category headers
)

# series: label ➜ list(row_indices)
bench_rows = all_benchmarks.groupby(all_benchmarks).apply(lambda x: x.index.tolist())

# how to aggregate duplicates?
handle_duplicates = "mean"        # ["mean", "first", "last", "max", "min"]

def aggregate(values):
    if handle_duplicates == "first":
        return values[0]
    if handle_duplicates == "last":
        return values[-1]
    if handle_duplicates == "max":
        return max(values)
    if handle_duplicates == "min":
        return min(values)
    return sum(values) / len(values)              # default: mean


# -------------------------------------------------
# 4.  Produce the string for every model column
# -------------------------------------------------
model_columns = df.columns[2:]                    # skip meta columns

model_strings = []

for col in model_columns:
    size, group = parse_model_name(col)

    # numeric version of the column
    numeric = pd.to_numeric(df[col], errors="coerce")

    # gather every benchmark score, keeping label unchanged
    scores = {}
    for label, idx_list in bench_rows.items():
        vals = numeric.loc[idx_list].dropna().tolist()
        if vals:                                  # at least one number present
            scores[label] = round(aggregate(vals), 2)

    # stringify
    score_blob = ",".join(f"{k}:{v}" for k, v in scores.items())
    model_strings.append(
        f'model:"{col}",size:{size},group:"{group}",scores:{{{score_blob}}}'
    )

# -------------------------------------------------
# 5.  Use the result
# -------------------------------------------------
for s in model_strings:
    print(s.replace("'", '')#.replace('AIME 2024', 'AIME24').replace('AIME2025', 'AIME25')
          )


model:"Final eval (Instruct)",size:None,group:"Final",scores:{}
model:"Qwen3-32B",size:32.0,group:"Qwen3",scores:{AGIEval:23.38,AIME24:27.71,AIME25:19.79,AMC23:67.34,ARC-C:48.98,Alpaca-Eval:64.21,BBH:62.47,DROP:10.26,GPQA:30.2,GPQA_Diamond:49.49,GSM8k:88.78,HellaSwag:68.89,HumanEval:90.85,HumanEval+:85.37,IFEval_inst:89.09,IFEval_prompt:84.84,LiveBench:63.05,LiveCodeBench:45.01,MATH (hendrycks):0.04,MATH-500:82.0,MBPP:86.24,MBPP+:71.96,MMLU:62.68,MMLU-Pro:54.68,MMLU-stem:58.56,Math-Verify:58.53,PIQA:79.6,TruthfulQA:58.58}
model:"Qwen2.5 72B",size:72.0,group:"Qwen2.5",scores:{AIME24:17.29,AIME25:15.21,AMC23:67.34,ARC-C:46.59,Alpaca-Eval:49.29,BBH:72.52,GPQA:37.67,GPQA_Diamond:44.95,GSM8k:82.26,HellaSwag:68.79,HumanEval:87.2,HumanEval+:80.49,IFEval_inst:88.97,IFEval_prompt:83.73,LiveBench:54.03,LiveCodeBench:54.6,MATH-500:83.6,MBPP:89.68,MBPP+:75.4,MMLU:83.5,MMLU-Pro:56.35,MMLU-stem:80.73,MTBench:9.16,Math-Verify:58.61,TruthfulQA:69.8}
model:"Qwen2.5 32B",size:32.0,group:"Qwen2.5",scores

In [8]:
import re
import pandas as pd

# -------------------------------------------
# 1.  Load or reuse your existing dataframe
# -------------------------------------------
# df = pd.read_csv("your_file.csv")           # ⬅ if coming from CSV
# (or) df = your_dataframe_variable           # ⬅ if it’s already in memory

# --------------------------------------------------------------------
# 2.  Helper to pull size (# of B or M) and model‑family (“group”) info
# --------------------------------------------------------------------
def parse_model_name(name: str):
    # normalise whitespace
    name = name.strip().replace("  ", " ")

    # ----- size -----
    m = re.search(r"([\d.]+)\s*([BM])", name, re.I)
    if m:
        num, unit = m.groups()
        size = float(num)
        if unit.upper() == "M":          # convert M → B so everything is in billions
            size /= 1_000
    else:
        size = None                      # leave as None if pattern is missing

    # ----- group / family -----
    # “Falcon-H1-34B”   ➜ “Falcon-H1”
    # “Qwen2.5 72B”     ➜ “Qwen2.5”
    group = re.split(r"\s|-(?=\d)", name)[0]   # stop before the first digit/space

    return size, group


# ----------------------------------------------------------
# 3.  Pick which rows correspond to the single‑task scores
#     you want to surface (feel free to tweak these labels)
# ----------------------------------------------------------
TASK_ROWS = {
    "MMLU":  "MMLU",      # label seen in first column
    "ARC":   "ARC-C"      # row label for ARC‑Challenge (zero‑shot)
}

# Quick look‑up Series:  label ➜ row‑index
task_idx = {k: int(df[df.iloc[:, 0] == v].index[0]) for k, v in TASK_ROWS.items()}

# ----------------------------------------------------------
# 4.  Assemble the dictionary & stringify it
# ----------------------------------------------------------
first_col, shots_col = df.columns[:2]           # easier handles

model_columns = [c for c in df.columns[2:]      # skip first two meta‑columns
                 if not c.lower().startswith("final")]  # ignore helper cols

model_strings = []

for col in model_columns:
    size, group = parse_model_name(col)

    # numeric scores for that model
    col_numeric = pd.to_numeric(df[col], errors="coerce")

    scores = {
        "Avg":  round(col_numeric.mean(skipna=True), 2),
        "MMLU": round(col_numeric.iloc[task_idx["MMLU"]], 2)
                 if pd.notna(col_numeric.iloc[task_idx["MMLU"]]) else None,
        "ARC":  round(col_numeric.iloc[task_idx["ARC"]], 2)
                 if pd.notna(col_numeric.iloc[task_idx["ARC"]]) else None,
    }

    # keep only the non‑missing entries inside the braces
    score_blob = ",".join(f"{k}:{v}" for k, v in scores.items() if v is not None)

    model_strings.append(
        f'model:"{col}",size:{size},group:"{group}",scores:{{{score_blob}}}'
    )

# -------------------------------------------
# 5.  Consume the output however you like
# -------------------------------------------
for s in model_strings:
    print(s)


model:"Qwen3-32B",size:32.0,group:"Qwen3",scores:{Avg:58.46,MMLU:80.89,ARC:48.98}
model:"Qwen2.5 72B",size:72.0,group:"Qwen2.5",scores:{Avg:63.78,MMLU:84.42,ARC:46.59}
model:"Qwen2.5 32B",size:32.0,group:"Qwen2.5",scores:{Avg:62.35,MMLU:82.8,ARC:44.54}
model:"Mistral-Small-3.1-24B",size:24.0,group:"Mistral-Small",scores:{Avg:77.17,MMLU:81.14}
model:"Gemma3-27B",size:27.0,group:"Gemma3",scores:{Avg:64.14,MMLU:78.01,ARC:54.52}
model:"Llama3.1 70B",size:70.0,group:"Llama3.1",scores:{Avg:60.3,MMLU:83.19,ARC:63.65}
model:"Llama4-scout",size:None,group:"Llama4-scout",scores:{Avg:61.31,MMLU:80.4,ARC:56.14}
model:"Falcon3.1 74B",size:74.0,group:"Falcon3.1",scores:{Avg:63.43,MMLU:82.33,ARC:63.4}
model:"GTP4o (mini)",size:None,group:"GTP4o",scores:{Avg:nan}
model:"gpt-4o-mini-2024-07-18",size:None,group:"gpt",scores:{Avg:nan}
model:"Falcon-H1-34B-16k",size:34.0,group:"Falcon-H1",scores:{Avg:66.95,MMLU:84.18,ARC:61.26}
model:"Falcon-H1-34B-128k",size:34.0,group:"Falcon-H1",scores:{Avg:67.09,MMLU: